<a href="https://colab.research.google.com/github/soto-xs/Universe_web/blob/main/Robot_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paso 1: **Configuración del Ambiente**
En este paso, se configuran las variables globales y se verifica que las bibliotecas necesarias estén instaladas:


In [3]:
# Instalamos las bibliotecas necesarias en Colab
!pip install yfinance beautifulsoup4 pandas numpy matplotlib requests

# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from IPython.display import clear_output
import time

# Definimos las variables globales
global df_bitcoin, precio_actual, tendencia, media_bitcoin, algoritmo_decision

# Verificamos que las bibliotecas estén instaladas correctamente
!python -V
print('------')
!pip show pandas | grep 'Name\|Version'
print('------')
!pip show numpy | grep 'Name\|Version'
print('------')
!pip show matplotlib | grep 'Name\|Version'


Python 3.10.12
------
Name: pandas
Version: 2.1.4
------
Name: numpy
Version: 1.26.4
        Name: lapack-lite
        Name: tempita
        Name: dragon4
        Name: libdivide
        Name: Meson
        Name: spin
        Name: OpenBLAS
        Name: LAPACK
        Name: GCC runtime library
        Version 3.1, 31 March 2009
                               Version 3, 29 June 2007
          5. Conveying Modified Source Versions.
          14. Revised Versions of this License.
        Name: libquadmath
------
Name: matplotlib
Version: 3.7.1


Paso 2: **Obtención de datos**
Aquí se extraen los datos históricos de Bitcoin usando yfinance y se realiza el Web Scraping de CoinMarketCap para obtener el precio y la tendencia.


In [1]:
# Función para importar los datos históricos de Bitcoin
def importar_base_bitcoin():
    global df_bitcoin
    # Obtenemos los últimos 7 días de datos a intervalos de 5 minutos
    df_bitcoin = yf.download('BTC-USD', interval='5m', period='7d')
    print("Datos de Bitcoin importados correctamente.")

# Función para extraer el precio actual y la tendencia usando web scraping
def extraer_tendencias():
    global precio_actual, tendencia
    url = "https://coinmarketcap.com/currencies/bitcoin/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extraer el precio actual de Bitcoin
    price_tag = soup.find('div', class_='priceValue').text
    precio_actual = float(price_tag.replace('$', '').replace(',', ''))

    # Extraer la variación en la última hora
    variation_tag = soup.find('span', {'data-timespan': '1h'}).text
    variation = float(variation_tag.replace('%', ''))

    # Determinamos la tendencia
    if variation < 0:
        tendencia = 'baja'
    else:
        tendencia = 'alta'

    print(f"Precio actual: {precio_actual} USD, Tendencia: {tendencia}")



Paso 3: **Limpieza de datos**
En esta parte, se limpia el DataFrame df_bitcoin, eliminando duplicados, valores nulos y outliers.

In [4]:
# Función para limpiar los datos
def limpieza_datos():
    global df_bitcoin, media_bitcoin
    # Creamos una copia de los datos
    df_bitcoin_limpio = df_bitcoin.copy()

    # Eliminamos duplicados en el índice
    df_bitcoin_limpio = df_bitcoin_limpio[~df_bitcoin_limpio.index.duplicated()]

    # Tratamos los valores nulos en la columna 'Close'
    df_bitcoin_limpio['Close'].fillna(method='ffill', inplace=True)

    # Eliminamos registros con volumen igual a 0
    df_bitcoin_limpio = df_bitcoin_limpio[df_bitcoin_limpio['Volume'] > 0]

    # Identificamos y eliminamos outliers usando boxplot
    Q1 = df_bitcoin_limpio['Close'].quantile(0.25)
    Q3 = df_bitcoin_limpio['Close'].quantile(0.75)
    IQR = Q3 - Q1
    df_bitcoin_limpio = df_bitcoin_limpio[(df_bitcoin_limpio['Close'] >= (Q1 - 1.5 * IQR)) & (df_bitcoin_limpio['Close'] <= (Q3 + 1.5 * IQR))]

    # Calculamos el precio promedio
    media_bitcoin = df_bitcoin_limpio['Close'].mean()
    print(f"Precio promedio del Bitcoin (limpio): {media_bitcoin}")


Paso 4: **Tomar decisiones**
Se decide si vender o comprar basándose en el precio actual, la media, y la tendencia.

In [5]:
# Función para tomar decisiones de compra o venta
def tomar_decisiones():
    global algoritmo_decision
    # Si el precio actual >= precio promedio y la tendencia es de baja, vender
    if precio_actual >= media_bitcoin and tendencia == 'baja':
        algoritmo_decision = 'Vender'
    # Si el precio actual < precio promedio y la tendencia es de alta, comprar
    elif precio_actual < media_bitcoin and tendencia == 'alta':
        algoritmo_decision = 'Comprar'
    else:
        algoritmo_decision = 'Esperar'

    print(f"Decisión del algoritmo: {algoritmo_decision}")


Paso 5: **Visualización**
Se genera el gráfico que muestra el precio y la decisión del algoritmo.

In [6]:
# Función para visualizar el gráfico
def visualizacion():
    global df_bitcoin, media_bitcoin, algoritmo_decision
    # Añadimos una columna de promedio al DataFrame
    df_bitcoin['Promedio'] = media_bitcoin

    # Configuramos el tamaño del gráfico
    plt.figure(figsize=(16, 5))

    # Título
    plt.title(f"Evolución del precio de Bitcoin y decisión: {algoritmo_decision}")

    # Gráfico de la columna 'Close' y el promedio
    plt.plot(df_bitcoin.index, df_bitcoin['Close'], label='Precio Bitcoin (Close)')
    plt.plot(df_bitcoin.index, df_bitcoin['Promedio'], label='Precio Promedio', linestyle='--')

    # Añadimos la decisión del algoritmo en el gráfico
    plt.annotate(algoritmo_decision, xy=(df_bitcoin.index[-1], df_bitcoin['Close'][-1]),
                 xytext=(df_bitcoin.index[-1], df_bitcoin['Close'].max()),
                 arrowprops=dict(facecolor='black', shrink=0.05))

    # Mostramos la leyenda y el gráfico
    plt.legend()
    plt.show()


Paso 6: **Automatización**
Este es el bucle infinito que actualiza el bot cada 5 minutos, limpiando los datos, tomando decisiones y actualizando el gráfico.

In [ ]:
from IPython.display import clear_output
import time
while(True):
  clear_output()
  importar_base_bitcoin()
  extraer_tendencias()
  limpieza_datos()
  tomar_decisiones()
  visualizacion()
  time.sleep(300)

[*********************100%***********************]  1 of 1 completed


KeyboardInterrupt: 